In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt

from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

file_path = 'USA_Lanker-1_1_T-1.xml'

scenario, planning_problem_set = CommonRoadFileReader(file_path).open()

plt.figure(figsize=(25, 10))
draw_object(scenario)
draw_object(planning_problem_set)
plt.gca().set_aspect('equal')
plt.show()


In [ ]:
draw_parameters = {'time_begin': 10}

plt.figure(figsize=(25, 10))
draw_object(scenario, draw_params=draw_parameters)
draw_object(planning_problem_set)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
import numpy as np

translation = np.array([0.0, 0.0])
angle = np.pi/2

scenario.translate_rotate(translation, angle)
planning_problem_set.translate_rotate(translation, angle)

plt.figure(figsize=(25, 10))
draw_object(scenario)
draw_object(planning_problem_set)
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
from commonroad.planning.goal import GoalRegion
from commonroad.common.util import Interval, AngleInterval
from commonroad.scenario.trajectory import State

# define example goal region
goal_state_1 = State(time_step=Interval(3, 5), orientation=AngleInterval(0.1, 1), velocity=Interval(20, 30.5))
goal_state_2 = State(time_step=Interval(3, 6), orientation=AngleInterval(0.1, 1), velocity=Interval(15, 25.5))

goal_region = GoalRegion([goal_state_1, goal_state_2])

# state of the ego vehicle
state = State(time_step=3, orientation=0.5, velocity=25)

# check if the state is inside of the goal region
goal_region.is_reached(state)

In [ ]:
from commonroad.common.file_writer import CommonRoadFileWriter
from commonroad.common.file_writer import OverwriteExistingFile

fw = CommonRoadFileWriter(scenario, planning_problem_set, "author", "affiliation", "source", "tags")

filename = "filename.xml"

fw.write_to_file(filename, OverwriteExistingFile.ALWAYS)

In [ ]:
import os

from commonroad.common.solution_writer import CommonRoadSolutionWriter, VehicleModel, VehicleType, CostFunction
from commonroad.scenario.trajectory import Trajectory, State

# prepare trajectory
pm_state_list = list()
for i in range(10):
    pm_state_list.append(State(**{'position': np.array([i, -i]), 'velocity': i*.2, 'velocity_y': i*0.001, 'time_step': i}))
trajectory_pm = Trajectory(0, pm_state_list)

# prepare control input vector (list of [x_acceleration, y_acceleration, time])
pm_input_list = np.array([[1.0, 3.5, 0.0], [2.0, 2.5, 0.1], [3.0, 1.5, 0.2]])

# write solution to a xml file
csw = CommonRoadSolutionWriter(output_dir=os.getcwd(), scenario_id='test_scenario', step_size=0.1,
                               vehicle_type=VehicleType.BMW_320i, vehicle_model=VehicleModel.PM,
                               cost_function=CostFunction.SA1)
# add trajectory solution
csw.add_solution_trajectory(trajectory_pm, planning_problem_id=5)

# or add control vector solution
csw.add_solution_input_vector(pm_input_list, planning_problem_id=8)

csw.write_to_file()